# Plot XRD maps from flyscans

This notebook will plot an XRD map from a flyscan. It requires data from the `.h5` masterfile and the `_pilatus_integrated.h5` file.

A ROI for the XRD map (in radial bins) should be defined in one of the cell below.

### Load data and plot average XRD for the map

In [ ]:
%matplotlib widget
import os
import sys
import h5py
import numpy as np
import matplotlib.pyplot as plt
#To import DanMAX from the folder above:
sys.path.append('../')
import DanMAX as DM
style = DM.darkMode(style_dic={'figure.figsize':'large'})

In [ ]:
# Define scan location:
#Note that scans must be a list!
scans = [XXX]
proposal,visit=DM.getCurrentProposal()

#Select ranges to load (in the unit the data were integrated with)
#useful for reducing the size of large datasets
xrd_range = None
azi_range = None

#Load data
maps = DM.mapping.stitchScans(scans,XRF=False,proposal=proposal,visit=visit,xrd_range=xrd_range,azi_range=azi_range)
#Apply I0 correction
xrd_map = (maps['xrd_map'].transpose(2,0,1)/ maps['I0_map']).transpose(1,2,0)
xrd_avg = np.mean(maps['xrd_map'],axis=(0,1))


### Plot avarage XRD map

In [ ]:
#Set ROIs:

# define ROI dictionary with a label str and a tuple of lower and upper limits, given in the scattering units (2theta or Q)
#     'label': (lower,upper)
xrd_rois = {'peak_1' : (  9.7, 9.9),
        'peak_2' : (15.88, 16.18)}

# plot average XRD pattern
plt.figure()
plt.title('Average XRD pattern')
plt.plot(maps['x_xrd'],xrd_avg)
print(maps['xrd_map'].dtype)
if maps['Q']:
    plt.xlabel('Q (A-1)')
else:
    plt.xlabel('2theta (deg)')
plt.ylabel('Intensity')
plt.yscale('log')

for key in xrd_rois:
    xrd_roi = (maps['x_xrd'] > xrd_rois[key][0]) & (maps['x_xrd'] < xrd_rois[key][1])
    plt.axvline(xrd_rois[key][0],c='k',ls='--',lw=1)
    plt.axvline(xrd_rois[key][1],c='k',ls='--',lw=1)
    plt.plot(maps['x_xrd'][xrd_roi],xrd_avg[xrd_roi], '.',ms=2,label=key)
plt.legend()

### Plot XRD map
It is probably nescessesary to change the vmax value

In [ ]:
# Set the number of columns for the figure
cols = 2

rows = int(len(xrd_rois)/cols) + (len(xrd_rois)%cols!=0)
# initialize subplots with shared x- and y-axes
fig,axs = plt.subplots(rows,cols,sharex=True,sharey=True)
fig.tight_layout()
axs = axs.flatten() # flatten the axes list to make it easier to index

I_xrd = []
for i,key in enumerate(xrd_rois):
    lower, upper = xrd_rois[key]
    lower, upper = np.digitize(lower, maps['x_xrd']), np.digitize(upper, maps['x_xrd'])
    
    #calculate background
    bkg = np.mean([np.mean(xrd_map[:,:,lower-5:lower],axis=2), np.mean(xrd_map[:,:,upper+1:upper+6],axis=2)],axis=0)
    # calculate the average diffraction peak signal from the integrated XRD data
    xrd_map_bgr = np.trapz((xrd_map[:,:,lower:upper].transpose(2,0,1) - bkg).transpose(1,2,0),maps['x_xrd'][lower:upper])
    xrd_map_bgr -=np.nanmin(xrd_map_bgr)
                           
    I_xrd.append(xrd_map_bgr)
    

    # plot the map as a pseudo colormesh
    ax = axs[i]
    ax.set_title(key)
    pcm = ax.pcolormesh(maps['x_map'],
                        maps['y_map'],
                        I_xrd[i],
                        shading='nearest')
    fig.colorbar(pcm,ax=ax,aspect=40,pad=0.05)
    ax.set_xlabel('x mm')
    ax.set_ylabel('y mm')
    # set the aspect ratio to equal to give square pixels
    ax.set_aspect('equal')

# delete surplus plots
for i in range(1,cols*rows-len(xrd_rois)+1):
    fig.delaxes(axs[-i])